In [1]:
import os
import pandas as pd
from tqdm import tqdm
import torch
import numpy as np
import re
from transformers import BertTokenizer, BertModel
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
os.chdir('C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\전처리데이터\\MBTI')

### Tokenize

In [20]:
# tqdm을 pandas에 적용하기 위한 설정
tqdm.pandas()

# BERT 토크나이저 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 예시 데이터 (실제 데이터로 교체 필요)
df = pd.read_csv('MBTI_prepro2.5.csv')

# 샘플 텍스트
text = df['posts_preprocessed']


# 토크나이징 및 입력 준비를 위한 함수 정의
def encode_text(text):
    inputs = tokenizer(text, 
                       return_tensors="pt", 
                       max_length=512, 
                       truncation=True, 
                       padding='max_length', 
                       add_special_tokens=True)
    with torch.inference_mode():
        outputs = model(**inputs)
    cls_vector = outputs.last_hidden_state[:, 0, :].numpy()
    return cls_vector

# tqdm을 사용하여 DataFrame의 각 행에 대해 함수 적용
cls_vectors = text.progress_apply(encode_text)

# 결과를 NumPy 배열로 변환
cls_vectors_np = np.array(cls_vectors.tolist())

# CLS 벡터를 NumPy 배열로 저장
np.save("cls_vector.npy", cls_vectors_np)


100%|██████████| 8675/8675 [2:32:47<00:00,  1.06s/it]  


##### else

In [ ]:


# tqdm을 pandas에 적용하기 위한 설정
tqdm.pandas()

# BERT 토크나이저 및 모델 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# GPU 사용 설정 (CUDA 사용 가능한 경우)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 예시 데이터 (실제 데이터로 교체 필요)
df = pd.read_csv('MBTI_prepro2.7.csv')

# 토크나이징 및 입력 준비를 위한 함수 정의
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding='max_length', add_special_tokens=True)
    inputs = inputs.to(device)  # 입력을 적절한 장치로 이동
    with torch.inference_mode():
        outputs = model(**inputs)
    cls_vector = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # 결과를 CPU로 이동 후 NumPy 배열로 변환
    return cls_vector

# 각 컬럼에 대해 CLS 벡터 추출 및 저장
for col in ['10', '20', '25', '30', '40']:
    print(f"Processing column: {col}")
    cls_vectors = df[col].progress_apply(encode_text)
    cls_vectors_np = np.vstack(cls_vectors)  # 리스트를 vstack을 사용하여 NumPy 배열로 변환
    np.save(f"{col}_cls.npy", cls_vectors_np)  # 저장


### text_preprocess & split sentence

In [34]:
df = pd.read_csv('C:\\Users\\david\\Desktop\\대학원\\Individual_project\\mbti_project\\MBTI&BigFive_data\\원본데이터\\mbti_rawdata.csv')
df.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
# 'posts' 컬럼을 ||| 기준으로 분할하고, 분할된 항목의 수를 세는 함수
df['count'] = df['posts'].apply(lambda x: len(x.split('|||')))

# 결과 확인
print(df[['posts', 'count']])

In [32]:
df['count'].sum()

422845

In [ ]:
# 'posts' 컬럼에서 문장을 분할하는 함수
def split_and_rejoin(row, num_sentences):
    sentences = row.split('|||')[:num_sentences]  # num_sentences 만큼 문장을 분할
    return '|||'.join(sentences)  # 다시 합쳐서 반환

# 각각의 문장 수에 대해 위의 함수를 적용하여 새 컬럼 생성
df['10sen'] = df['posts'].apply(lambda x: split_and_rejoin(x, 10))
df['20sen'] = df['posts'].apply(lambda x: split_and_rejoin(x, 20))
df['25sen'] = df['posts'].apply(lambda x: split_and_rejoin(x, 25))
df['30sen'] = df['posts'].apply(lambda x: split_and_rejoin(x, 30))
df['40sen'] = df['posts'].apply(lambda x: split_and_rejoin(x, 40))

# 결과 확인
df

In [36]:
def preprocess_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    
    # Remove special symbols and punctuation (excluding periods)
    text = re.sub(r'(?<!\.)\.{3,}(?!\.)', '', text)  # Remove occurrences of three or more periods
    text = re.sub(r'[^\w\s\.]', '', text)  # Remove special symbols and punctuation except periods
    
    # Remove all numbers (improved to catch more patterns)
    text = re.sub(r'\b\d+(\.\d+)?', '', text)  # Remove numbers (both integers and decimals)
    text = re.sub(r'\d+', '', text)  # Remove all remaining digits
    
    text = re.sub(r'_', '', text)  # Remove underscores
    text = text.lower()

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading and trailing spaces
    text = text.strip()
    
    return text

In [ ]:
df['10']= df['10sen'].apply(preprocess_text)
df['20']=df['20sen'].apply(preprocess_text)
df['25']=df['25sen'].apply(preprocess_text)
df['30']=df['30sen'].apply(preprocess_text)
df['40']=df['40sen'].apply(preprocess_text)
df

In [ ]:
dff = df.drop(columns=['posts', '10sen', '20sen','25sen', '30sen', '40sen'], inplace=True)
df

In [40]:
df.to_csv('MBTI_prepro2.7.csv', index=False, encoding= 'utf-8-sig')